In [1]:
import os
import sys
from pathlib import Path
import re
from tqdm import tqdm

In [ ]:
audio_files = Path("/data/tagesschau/more_data/")
files = sorted(list(audio_files.glob("**/*.webm")))
for path in tqdm(files):
    if not os.path.exists(os.path.join(path.parent / 'audio.wav')):
        # print(f"{path} ==> {os.path.join(path.parent / 'audio.wav')}")
        os.system(f"ffmpeg -i \"{path}\" -vn \"{os.path.join(path.parent / 'audio.wav')}\"")

In [33]:
counts = []
no_counts = []
for i in range(len(list(Path("/data/tagesschau/more_data/").glob("**/*.txt")))):
    if not os.path.exists(f"/data/tagesschau/more_data/{i}/{i}.txt"):
        continue
    with open(f"/data/tagesschau/more_data/{i}/{i}.txt", "r") as f:
        lines = f.read().splitlines()
    lines = [line.strip().replace("10.10", "10:10") for line in lines]
    labels = []
    for line in lines:
        if re.match(r"\d{1,2}:\d\d", line):
            search = re.search(r"^\d{1,2}:\d{1,2}", line)
            labels.append(search)
            time = line[search.span()[0]: search.span()[1]]
            m, s = time.split(":")
            # print(search, time, int(m), int(s), "\t", int(m)*60 + int(s))
    if len(labels) > 1:
        counts.append(i)
    else:
        no_counts.append(i)

In [38]:
print(len(counts))
print(len(no_counts))

927
1426


In [58]:
import pandas as pd

In [72]:
timestamps = {}
for count in counts:
    i = count
    timestamps[count] = {}
    with open(f"/data/tagesschau/more_data/{i}/{i}.txt", "r") as f:
        lines = f.read().splitlines()
    times = []
    for line in lines:
        if re.match(r"\d{1,2}:\d\d", line):
            search = re.search(r"^\d{1,2}:\d{1,2}", line)
            labels.append(search)
            time = line[search.span()[0]: search.span()[1]]
            m, s = time.split(":")
            times.append((int(m), int(s)))
    timestamps[count]["duration"] = times

In [70]:
import json

In [73]:
json.dump(timestamps, open("/data/tagesschau/more_data/labels.json", "w"))